In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# from degree_days import dds
import datetime
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse_fraction, contri, get_tensor, create_region_df_dfc_static
from create_matrix import *
from tensor_custom_core import *

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "Austin"
year = 2014

import os

def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [3]:
import sys
case=2
a = 3
b = 2
source = 'Austin'
target = 'SanDiego'
constant_use = 'True'
start = 1
stop = 13

In [4]:
source_df, source_dfc, source_tensor, source_static = create_region_df_dfc_static(source, year, start, stop)
target_df, target_dfc, target_tensor, target_static = create_region_df_dfc_static(target, year, start, stop)

# # using cosine similarity to compute L
source_L = get_L(source_static)
target_L = get_L(target_static)

# Seasonal constant constraints
if constant_use == 'True':
    T_constant = np.ones(stop-start).reshape(-1 , 1)
else:
    T_constant = None

In [5]:
tensor_copy = source_tensor.copy()
tensor_copy[:, 1:, :] = np.NaN

In [10]:
from degree_days import dds

#For HVAC

In [11]:
def get_tensor_appliance(df, dfc, appliance):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] ]).flatten()
    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 1, stop - start))
    M, N, O = tensor.shape
    return tensor

In [50]:
hvac_tensor = get_tensor_appliance(source_df, source_dfc, 'fridge')
agg_tensor = get_tensor_appliance(source_df, source_dfc, 'aggregate')
tensor = np.concatenate((agg_tensor, hvac_tensor), axis=1)

In [73]:
known_homes = []
random_list =  random.sample(range(num_train), 50)
random_list =[x+1 for x in random_list]

pos = random_list[:50]
known_homes = known_homes+pos
# aggregate readings for all homes are known


In [ ]:
def compute_rmse_fraction(appliance, pred_df, region='Austin', year=2014):
	appliance_df = create_matrix_region_appliance_year(region, year, appliance)

	if appliance == "hvac":
		start, stop = 5, 11
	else:
		start, stop = 1, 13
	pred_df = pred_df.copy()
	pred_df.columns = [['%s_%d' % (appliance, month) for month in range(start, stop)]]
	gt_df = appliance_df[pred_df.columns].ix[pred_df.index]

	aggregate_df = appliance_df.ix[pred_df.index][['aggregate_%d' % month for month in range(start, stop)]]

	aggregate_df.columns = gt_df.columns
	rows, cols = np.where((aggregate_df < 100))
	for r, c in zip(rows, cols):
		r_i, c_i = aggregate_df.index[r], aggregate_df.columns[c]
		aggregate_df.loc[r_i, c_i] = np.NaN

	gt_fraction = gt_df.div(aggregate_df) * 100
	pred_fraction = pred_df.div(aggregate_df) * 100

	# Capping it to 100%
	pred_fraction[pred_fraction > 100] = 100.

	gt_fraction_dropna = gt_fraction.unstack().dropna()
	pred_fraction_dropna = pred_fraction.unstack().dropna()
	index_intersection = gt_fraction_dropna.index.intersection(pred_fraction_dropna.index)
	gt_fraction_dropna = gt_fraction_dropna.ix[index_intersection]
	pred_fraction_dropna = pred_fraction_dropna.ix[index_intersection]
	difference_error = (gt_fraction_dropna - pred_fraction_dropna).abs()

	rms = np.sqrt(mean_squared_error(gt_fraction_dropna, pred_fraction_dropna))
	return gt_fraction_dropna, pred_fraction_dropna, rms, difference_error

In [83]:
from scipy.optimize import nnls
from sklearn.model_selection import train_test_split, KFold
import random

random_list =  random.sample(range(533), 533)
random_list =[x+1 for x in random_list]

n_splits = 10
case = 2
a = 3
b = 3
cost = 'abs'
app = 'fridge'
known_homes = []

   
kf = KFold(n_splits=n_splits)

for random_seed in range(1):
    
    random.seed(random_seed)
    
    for train_max, test in kf.split(source_df):
            
            num_train = len(train_max)
            num_test = len(test)

            train = train_max
           
            # get the index of training and testing data
            train_ix = source_df.index[train]
            test_ix = source_df.index[test]
            print "test_ix: ", test_ix

            # create the tensor
            train_test_ix = np.concatenate([train_ix, test_ix])
            df_t, dfc_t = source_df.ix[train_test_ix], source_dfc.ix[train_test_ix]
            app_tensor = get_tensor_appliance(df_t, dfc_t, app)
            agg_tensor = get_tensor_appliance(df_t, dfc_t, 'aggregate')
            
            tensor = np.concatenate((agg_tensor, app_tensor), axis=1)
            
            
            random_list =  random.sample(range(num_train), 50)
            random_list =[x+1 for x in random_list]

            for month in range(1):
                if month == 0:
                    tensor_copy = tensor.copy()
                    tensor_copy[:,:,:] = np.NaN
                    pos = random_list[:50]
                    known_homes = known_homes + pos
                    # aggregate readings for all homes are known
                    tensor_copy[:, 0, :] = tensor[:, 0, :]
                    # appliance readings for selected homes are known
                    tensor_copy[known_homes, 1, :] = tensor[known_homes, 1, :]
                    tensor_copy = tensor_copy[:, :, 0:(month+2)]
                    
                    T_constant = np.ones(month+2).reshape(-1,1)
                    H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                                                source_L,
                                                                              a,
                                                                              b,
                                                                              num_iter=1300,
                                                                              lr=0.1, dis=True,
                                                                              lam=0,
                                                                              T_known=T_constant)
                else:
                    # Generate tensor_copy
                    tensor_copy = tensor.copy()
                    tensor_copy[:,:,:] = np.NaN
                    known_homes = known_homes + pos
                    # aggregate readings for all homes are known
                    tensor_copy[:, 0, :] = tensor[:, 0, :]
                    # appliance readings for selected homes are known
                    tensor_copy[known_homes, 1, :] = tensor[known_homes, 1, :]
                    tensor_copy = tensor_copy[:, :, 0:(month+2)]
                    
                    ####
                    T_constant = np.ones(month+2).reshape(-1,1)
                    H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                                                source_L,
                                                                              a,
                                                                              b,
                                                                              num_iter=1300,
                                                                              lr=0.1, dis=True,
                                                                              lam=0,
                                                                              T_known=T_constant)
                
                ## Get the home ids
                ### Based on the Home factors
                HAT = multiply_case(H, A, T, case)
                
                pred_test = pd.DataFrame(HAT[num_train:, 1, :], index=test_ix)
                rms_test, error_test = compute_rmse_fraction(app, pred_test, 'Austin', 1, month+3)[2:]
                pred_train = pd.DataFrame(HAT[:num_train, 1, :], index=train_ix)
                rms_train, error_train = compute_rmse_fraction(app, pred_train, 'Austin', 1, month+3)[2:]
                
                
                H_train = H[:num_train]
                H_train = H_train/np.max(H_train)
                label = KMeans(n_clusters=10, random_state=0).fit_predict(H_train)
                
                error_home = pd.concat([error_train[appliance + "_{}".format(start)], 
                   error_train[appliance + "_{}".format(start+1)]],axis=1)
                for i in range(start+2, end):
                    error_home = pd.concat([error_home, error_train[appliance + "_{}".format(i)]], axis = 1)
                app = np.sqrt((error_home**2).mean(axis=1))
                
                max_error = 0
                max_cluster = -1
                for i in range(10):
                    index_set = find(label, i)
                    ids = train_ix[k]
                    mean = cluster_mean_error(app, ids)
                    if mean > max_error:
                        max_error = mean
                        max_cluster = i
                



test_ix:  Int64Index([ 22,  26,  48,  59,  68,  77,  86,  93,  94, 101, 114, 115, 121,
            130, 135, 160, 171, 187, 222, 243, 252, 267, 297, 347, 364, 370,
            410, 434, 436, 457, 470, 484, 491, 499, 503, 507, 508, 545, 555,
            580, 585, 624, 645, 661, 668, 739, 744, 772, 781, 861, 871, 878,
            890, 898],
           dtype='int64')
670.983049446
111.941447556
40.8817683068
test_ix:  Int64Index([ 936,  946,  954,  974,  980,  994, 1037, 1069, 1086, 1103, 1105,
            1153, 1169, 1185, 1192, 1202, 1283, 1310, 1314, 1331, 1334, 1403,
            1415, 1463, 1464, 1479, 1500, 1507, 1508, 1551, 1577, 1586, 1589,
            1601, 1617, 1632, 1642, 1681, 1697, 1700, 1714, 1718, 1782, 1790,
            1791, 1792, 1796, 1800, 1801, 1854, 1889, 1947, 1953, 1994],
           dtype='int64')
653.576385008


KeyboardInterrupt: 

In [119]:
train_ix[0]

22

In [91]:
def find(lst, value):
    return [i for i, x in enumerate(lst) if x==value]

In [175]:
from sklearn.cluster import KMeans
H_train = H[:num_train]
print np.max(H_train)
H_train = H_train/np.max(H_train)
label = KMeans(n_clusters=10, random_state=0).fit_predict(H_train)
k = find(label,5)
print k


10.8433883752
[0, 2, 9, 10, 20, 25, 64, 71, 106, 110, 125, 184, 193, 219, 227, 238, 240, 256, 262, 287, 292, 294, 321, 341, 342, 345, 349, 357, 370, 379, 392, 409, 411, 412, 414, 421, 422, 446, 448, 450, 463, 466, 478]


In [176]:
def cluster_mean_error(app_error, index):
    sum_error = 0
    for i in index:
#         print app_error[i]
        if i in app_error.index:
            sum_error += app_error[i]
    return sum_error/len(index)

In [177]:
start = 1
end = 3
appliance = 'fridge'
error_home = pd.concat([error_train[appliance + "_{}".format(start)], 
                   error_train[appliance + "_{}".format(start+1)]],axis=1)

for i in range(start+2, end):
    error_home = pd.concat([error_home, error_train[appliance + "_{}".format(i)]], axis = 1)
app = np.sqrt((error_home**2).mean(axis=1))

ids = train_ix[k]
mean = cluster_mean_error(app, ids)
print mean

# app.index
# app[936]

5.63938537858


In [46]:
def compute_error(HAT, index, domain):
    pred = {}
    for appliance_name, appliance_id in appliance_index.iteritems():
        pred[appliance_name] = []
        pred[appliance_name].append(pd.DataFrame(HAT[:,appliance_id,:], index=index))
#     print pred
    for appliance_name, appliance_id in appliance_index.iteritems():
        pred[appliance_name] = pd.concat(pred[appliance_name]).ix[index]
    
    
#     pred = pd.DataFrame(pred)
    err_all = {}
    err = {}
    for appliance in APPLIANCES_ORDER[1:]:        
        if appliance=="hvac":
            err_all[appliance], err[appliance] = compute_rmse_fraction(appliance, pred[appliance][range(4, 10)], domain)[2:]
        else:   
            err_all[appliance], err[appliance] = compute_rmse_fraction(appliance, pred[appliance], domain)[2:]

    err_app = {}
    for appliance in APPLIANCES_ORDER[1:]:
        if appliance == 'hvac':
            start, end = 5, 11
        else:
            start, end = 1, 13

        error_home = pd.concat([err[appliance][appliance + "_{}".format(start)], 
                           err[appliance][appliance + "_{}".format(start+1)]],axis=1)

        for i in range(start+2, end):
            error_home = pd.concat([error_home, err[appliance][appliance + "_{}".format(i)]], axis = 1)
        app = np.sqrt((error_home**2).mean(axis=1))
        err_app[appliance] = app

    result = (pd.DataFrame(err_app).fillna(0)*pd.Series(contri[domain])).sum(axis=1)
        
        
    return err_all, result

In [78]:
H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=True,
                                                          lam=0,
                                                          T_known=T_constant)

927.359618339
327.132788609
174.115738163


In [79]:
H

array([[ 1.66345507,  1.8757803 ,  1.69049574],
       [ 5.56361928,  5.50529646,  5.50663552],
       [ 1.3197905 ,  1.78687358,  1.804727  ],
       ..., 
       [ 2.73141718,  2.82475159,  2.26333804],
       [ 2.24214654,  2.14861078,  2.76485631],
       [ 2.99715369,  3.21232919,  3.28624526]])

In [11]:
err_all, err_home = compute_error(HATs[-1], source_df.index, source)

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


hvac
fridge
mw
dw
wm
oven


In [45]:
source_df.ix[max_index]['aggregate_3']

77.594146728515625

In [17]:
k = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()# error = (pd.DataFrame(out)*pd.Series(contri['SanDiego'])).sum(axis=1)

In [21]:
max_index = err_home.argmax()

In [40]:
position = source_df.index.get_loc(max_index)

In [42]:
tensor_copy[position] = source_tensor[position]

In [74]:
import random
random.seed(5)
random_list =  random.sample(range(533), 533)
random_list =[x+1 for x in random_list]

In [77]:
prediction_error = []

tensor_copy = source_tensor.copy()
tensor_copy[:, 1:, :] = np.NaN
H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=False,
                                                          lam=0,
                                                      T_known=T_constant)
err_all, err_home = compute_error(HATs[-1], source_df.index, source)
overall_error = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()
print overall_error

for i in range(20):
    max_index = err_home.argmax()
    position = source_df.index.get_loc(max_index)
#     position = random_list[i]
    print position
    tensor_copy[position] = source_tensor[position]
    
    H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=False,
                                                          lam=0,
                                                          T_known=T_constant)
    err_all, err_home = compute_error(HATs[-1], source_df.index, source)
    overall_error = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()
    print i, overall_error

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


26.1021914905
61
0 22.1222043053
47
1 19.8106416228
476
2 18.4501308997
178
3 16.5123081787
518
4 17.1332797771
494
5 17.0051122383
45
6 16.8313282303
318
7 14.7923720999
131
8 14.7421349938
346
9 14.6333198034
153
10 13.6021757488
59
11 13.596117084
439
12 13.4803772491
425
13 13.5001449042
494
14 13.5001449042
494
15 13.5001449042
494
16 13.5001449042
494
17 13.5001449042
494
18 13.5001449042
494
19 13.5001449042
